# Basic Settings
---

## 1. Import Necessary Modules

In [1]:
import json
import requests
import datetime
import time
import numpy as np
import pandas as pd
import os

## 2. Set your FRED API Key

In [ ]:
# # Load the secret api key if it exists in terms of json.
# with open('secrets_file.json', 'r') as f:
#     secrets = json.load(f)
    
# print(secrets.keys())

In [2]:
# Or enter your API key directly.
api_key = ''

## 3. Setting a Class to interact with the FRED API

In [34]:
# class that will interact with the FRED API
class FredPy:
    def __init__(self, token = None):
        self.token = token
        self.url = "https://api.stlouisfed.org/fred/series/observations" + \
                    "?series_id={seriesID}&api_key={key}&file_type=json" + \
                    "&observation_start={start}&observation_end={end}&units={units}" + \
                    "&frequency={frequency}"
        
    def set_token(self, token):
        self.token = token
        
    def get_series(self, seriesID, start, end, units, frequency, name):
        date_temp = datetime.date(int(start[0:4]), int(start[5:7]), int(start[8:]))
        start_temp = str(date_temp)
        
        # The URL string with the values inserted into it
        url_formatted = self.url.format(
            seriesID = seriesID, start = start_temp, end = end,
            units = units, frequency = frequency, key = self.token
            )
        
        response = requests.get(url_formatted)

        if(self.token):
            # If the response was successful, extract the data from it.
            if(response.status_code == 200): # If 200, successful!
                # Loading contents
                # designate for the datatype to be correct using 'assign'
                # missing values replaced with their previous values
                data = pd.DataFrame(response.json()['observations'])[['date', 'value']]\
                .replace(r'^.$', np.NAN, regex = True)\
                .fillna(method = 'ffill')\
                .assign(date = lambda cols: pd.to_datetime(cols['date']))\
                .assign(value_2 = lambda cols: cols['value'].astype(float))\
                .rename(columns = {'value_2': name})

                return data[['date', name]]
            else:
                raise Exception(f"Bad response from API, " + \
                                 "status code = {response.status_code}")
        else:
            raise Exception(f"You did not specify an API key.")

In [38]:
# Instantiate fredpy object
fredpy = FredPy()

# Set the API key
fredpy.set_token(api_key)

In [39]:
# Settings for time-series
start_date = '2010-01-01'
end_date = str(datetime.date.today())
# Refer to the following URL for units.
# https://fred.stlouisfed.org/docs/api/fred/series_observations.html#units
units = 'lin'
frequency = 'm'

# Loading currency data
---

## 1. Loading FRED Currency Data

In [6]:
# Settings for currency table
currencies = {'USDCAD': 'EXCAUS', 'USDMXN': 'EXMXUS',
              'EURUSD': 'EXUSEU', 'USDJPY': 'EXJPUS',
              'USDKRW': 'EXKOUS'
             }

In [7]:
# Getting the series you want
currency_temp = None

for i in currencies:
    name, seriesID = i, currencies[i]
    df_temp = fredpy.get_series(
                seriesID = seriesID,
                start = start_date,
                end = end_date,
                units = units,
                frequency = frequency,
                name = name
                )
    if currency_temp is not None:
        currency_temp = pd.merge(currency_temp, df_temp[['date', name]],
                        on = 'date', how = 'outer')
    else:
        currency_temp = df_temp
#     if currency is not None:
#         currency = pd.concat([currency, df_temp[name]], axis = 1)
#     else:
#         currency = df_temp

In [8]:
currency_temp['date'] = currency_temp['date'].dt.strftime('%Y-%m')
currency_temp

,date,USDCAD,USDMXN,EURUSD,USDJPY,USDKRW
1,2010-01,1.0438,12.8096,1.4266,91.1011,1138.1947
2,2010-02,1.0572,12.9396,1.3680,90.1395,1155.6553
3,2010-03,1.0229,12.5673,1.3570,90.7161,1136.0822
4,2010-04,1.0052,12.2396,1.3417,93.4527,1115.4591
5,2010-05,1.0403,12.7262,1.2563,91.9730,1164.8410
...,...,...,...,...,...,...
142,2021-10,1.2434,20.4399,1.1600,113.1215,1181.8900
143,2021-11,1.2567,20.8698,1.1416,113.9650,1184.0600
144,2021-12,1.2800,20.9260,1.1301,113.8329,1183.8871
145,2022-01,1.2622,20.5074,1.1317,114.8255,1196.0345


## 2. Obtaining the recent monthly average Using Yahoo Finance API

We obtain the recent month's average value via averaging the daily fx rates from the 1st day of the last month to the recent trading day. Values from non-trading days are not included in the calculation.

In [9]:
#pd.date_range(end = datetime.datetime.today(), periods = 100).to_pydatetime().tolist()
# or
start = str(datetime.date(int(end_date[0:4]),
                          int(end_date[5:7]),
                          int(1)))
interval = '1d'

date_series = pd.date_range(start = start, end = end_date).to_pydatetime().tolist()
recent_temp = pd.DataFrame(date_series, columns = ['date'])

In [10]:
def query_builder(start, end, interval, ticker):
    query_string = f'https://query1.finance.yahoo.com/v7/finance/download/' + \
                   f'{ticker}?period1={int(time.mktime(start.timetuple()))}&period2={int(time.mktime(end.timetuple()))}&interval={interval}' + \
                   f'&events=history&includeAdjustedClose=true'
    return query_string

In [11]:
# fx rates
currency_indices = {
    'USDCAD': 'CAD=X', 'USDMXN': 'MXN=X', 'EURUSD': 'EURUSD=X',
    'USDJPY': 'JPY=X', 'USDKRW': 'KRW=X'
}

In [12]:
start_temp = datetime.date(int(end_date[0:4]),
                           int(end_date[5:7]),
                           int(1))
end_temp = datetime.date(int(end_date[0:4]),
                         int(end_date[5:7]),
                         int(end_date[8:10]))

In [13]:
for i in currency_indices:
    ticker = currency_indices[i]
    query = query_builder(start_temp, end_temp, interval, ticker)
    df_temp = pd.read_csv(query)[['Date', 'Close']]\
    .rename(columns = {'Close': i}).rename(columns = {'Date': 'date'})\
    .assign(date = lambda cols: pd.to_datetime(cols['date']))
    recent_temp = pd.merge(recent_temp, df_temp[['date', i]],
                        on = 'date', how = 'outer')

In [14]:
recent_temp = recent_temp.append(
                                 recent_temp.mean(skipna = True).rename('temp')
)
recent_temp.iloc[-1, recent_temp.columns.get_loc('date')] = end_date

C:\Users\1suof\AppData\Local\Temp\ipykernel_24908\2071808639.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  recent_temp.mean(skipna = True).rename('temp')
C:\Users\1suof\AppData\Local\Temp\ipykernel_24908\2071808639.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recent_temp = recent_temp.append(


In [15]:
recent_temp['date'] = recent_temp['date'].dt.strftime('%Y-%m')
currency = currency_temp.append(recent_temp[-1:], ignore_index = True)

C:\Users\1suof\AppData\Local\Temp\ipykernel_24908\989372491.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  currency = currency_temp.append(recent_temp[-1:], ignore_index = True)


In [16]:
currency.rename(columns = {'date': 'month'}, inplace = True)
currency

,month,USDCAD,USDMXN,EURUSD,USDJPY,USDKRW
0,2010-01,1.043800,12.809600,1.426600,91.101100,1138.194700
1,2010-02,1.057200,12.939600,1.368000,90.139500,1155.655300
2,2010-03,1.022900,12.567300,1.357000,90.716100,1136.082200
3,2010-04,1.005200,12.239600,1.341700,93.452700,1115.459100
4,2010-05,1.040300,12.726200,1.256300,91.973000,1164.841000
...,...,...,...,...,...,...
142,2021-11,1.256700,20.869800,1.141600,113.965000,1184.060000
143,2021-12,1.280000,20.926000,1.130100,113.832900,1183.887100
144,2022-01,1.262200,20.507400,1.131700,114.825500,1196.034500
145,2022-02,1.271100,20.457800,1.134900,115.276300,1199.122600


In [ ]:
# Export the currency dataframe as a csv file.
path = os.getcwd()
path = path + '.\\data\\currency_as_of_' + end_date + '.csv'
currency.to_csv(path, index = False, header = True)

# Loading inflation index data
---

Please remember that seasonality is basically inherent in inflation rates. And inflation indices have trends so that you must remove trends from them before you start doing time-series analysis.

In [17]:
# Settings for currency table
inflation_indices = {'Canada': 'CPALCY01CAM661N', # CPI of all times in canada
                    'Mexico': 'MEXCPIALLMINMEI', # CPI of all times in Mexico
                    'Euro Area': 'CP0000EZ19M086NEST', # 19 countries, 
                    'Japan': 'JPNCPIALLMINMEI', # CPI of all times in Japan
                    'South Korea': 'KORCPIALLMINMEI' # CPI of all times in South Korea
                   }

In [18]:
inflation_temp = None

for i in inflation_indices:
    name, seriesID = i, inflation_indices[i]
    df_temp = fredpy.get_series(
                seriesID = seriesID,
                start = start_date,
                end = end_date,
                units = units,
                frequency = frequency,
                name = name
                )
    if inflation_temp is not None:
        inflation_temp = pd.merge(inflation_temp, df_temp[['date', name]],
                        on = 'date', how = 'outer')
    else:
        inflation_temp = df_temp

In [19]:
inflation_temp['date'] = inflation_temp['date'].dt.strftime('%Y-%m')
# The last row may have 'NaN' since the data are not available at the moment.
# index 2015 = 100 (That is, the average value of 12 indices in 2015)
inflation_temp.rename(columns = {'date': 'month'}, inplace = True)
inflation_temp

,month,Canada,Mexico,Euro Area,Japan,South Korea
1,2010-01,91.279109,82.770410,91.56,96.614915,89.974884
2,2010-02,91.439247,83.249135,91.85,96.614915,90.156202
3,2010-03,91.679456,83.840154,92.86,96.818529,90.338575
4,2010-04,91.919664,83.573013,93.24,96.818529,90.702265
5,2010-05,91.999733,83.046416,93.35,96.818529,90.793978
...,...,...,...,...,...,...
142,2021-10,112.977914,131.836824,109.41,101.705269,108.948979
143,2021-11,113.218122,133.346158,109.90,101.908883,109.497150
144,2021-12,113.138053,133.829875,110.37,101.908883,109.676360
145,2022-01,113.778608,134.621619,110.70,102.112497,110.361573


In [20]:
# Export the inflation dataframe as a csv file.
path = os.getcwd()
path = path + '.\\data\\inflation_as_of_' + end_date + '.csv'
inflation_temp.to_csv(path, index = False, header = True)

# Loading the US short-term interest rate
---

Replace the probability that Federal Reserve System's policy rate change with the US short-term interest rate due to data availability. The historical probability is not accessible for public users. Instead, we use the US short-term interest rate, 3-Month Treasury Bill Secondary Market Rate, as its proxy since short-term rates are very closely related to central banks' policy rates.

## 1. Loading FRED short term interest rate data

In [52]:
short_term_rate_temp = None

df_temp = fredpy.get_series(
    seriesID = 'TB3MS',
    start = start_date,
    end = end_date,
    units = units,
    frequency = frequency,
    name = '3M T-Bill'
)

if short_term_rate_temp is not None:
    short_term_rate_temp = pd.merge(short_term_rate_temp,
                                    df_temp[['date', name]],
                                    on = 'date', how = 'outer')
else:
    short_term_rate_temp = df_temp

In [53]:
short_term_rate_temp['date'] = short_term_rate_temp['date'].dt.strftime('%Y-%m')
short_term_rate_temp

,date,3M T-Bill
0,2010-01,0.06
1,2010-02,0.11
2,2010-03,0.15
3,2010-04,0.16
4,2010-05,0.16
...,...,...
141,2021-10,0.05
142,2021-11,0.05
143,2021-12,0.06
144,2022-01,0.15


We obtain the recent month's value via averaging the daily 3-Month Treasury Bill Secondary Market Rate from the 1st day of the last month to the recent trading day. Values from non-trading days are not included in the calculation.

## 2. Loading the recent month daily data and averaging

In [54]:
short_term_rate_temp2 = None

df_temp = fredpy.get_series(
    seriesID = 'DTB3',
    start = start,
    end = end_date,
    units = units,
    frequency = 'd',
    name = '3M T-Bill'
)

if short_term_rate_temp2 is not None:
    short_term_rate_temp2 = pd.merge(short_term_rate_temp2,
                                     df_temp[['date', name]],
                                     on = 'date', how = 'outer')
else:
    short_term_rate_temp2 = df_temp

In [55]:
short_term_rate_temp2 = short_term_rate_temp2.append(
    short_term_rate_temp2.mean(skipna = True).rename('temp')
)
short_term_rate_temp2.iloc[-1, short_term_rate_temp2.columns.get_loc('date')] = end_date

C:\Users\1suof\AppData\Local\Temp\ipykernel_24908\3405712893.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  short_term_rate_temp2.mean(skipna = True).rename('temp')
C:\Users\1suof\AppData\Local\Temp\ipykernel_24908\3405712893.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  short_term_rate_temp2 = short_term_rate_temp2.append(


In [56]:
short_term_rate_temp2['date'] = short_term_rate_temp2['date'].dt.strftime('%Y-%m')
short_term_rate = short_term_rate_temp.append(short_term_rate_temp2[-1:], ignore_index = True)

C:\Users\1suof\AppData\Local\Temp\ipykernel_24908\1429122190.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  short_term_rate = short_term_rate_temp.append(short_term_rate_temp2[-1:], ignore_index = True)


In [59]:
short_term_rate.rename(columns = {'date': 'month'}, inplace = True)
short_term_rate

,month,3M T-Bill
0,2010-01,0.060000
1,2010-02,0.110000
2,2010-03,0.150000
3,2010-04,0.160000
4,2010-05,0.160000
...,...,...
142,2021-11,0.050000
143,2021-12,0.060000
144,2022-01,0.150000
145,2022-02,0.330000


In [60]:
# Export the short-term interest rate dataframe as a csv file.
path = os.getcwd()
path = path + '.\\data\\short_term_rate_as_of_' + end_date + '.csv'
short_term_rate.to_csv(path, index = False, header = True)